
# ***lien Github*** : https://github.com/GhazwaMa/DM_Project

>  ***Ghazwa Machfar*** 3DNI1
>
>  **Projet Data Mining**


## collecte des tweets

In [7]:
from tweepy import OAuthHandler
from tweepy import API

# Consumer key authentication
auth = OAuthHandler("vdA0dkB3GxvhXHgeXMONmWYoH", "uvdUl6VZNhT1huqeYlE2HYmN57VaM6Rco4HpGbhwZ5Y2B7Ubyy")

# Access key authentication
auth.set_access_token("1331310694105559046-ShzOiLPVT7pT5vnQsGphlwPMSrEOSU", "CTFsbUPR6mv8adDrImixmnfDOT0M6I9YgQgoo9AprIgTJ")

# Set up the API with the authentication handler
api = API(auth)


In [10]:
from tweepy.streaming import StreamListener
import json
import time
import sys

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 


In [13]:
from tweepy import Stream

# Set up words to track
keywords_to_track = ["hello","you"]

# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data
stream.filter(track = keywords_to_track)

KeyboardInterrupt: 

In [28]:
import glob
tweets = []
files  = list(glob.iglob('streamer_*.json'))
for f in files:
    fh = open(f, 'r', encoding = 'utf-8')
    tweets_json = fh.read().split("\n")

    ## remove empty lines
    tweets_json = list(filter(len, tweets_json))

    ## parse each tweet
    for tweet in tweets_json:
        try:
            tweet_obj = json.loads(tweet)

            ## flatten the file to include quoted status and retweeted status info
            if 'quoted_status' in tweet_obj:
                tweet_obj['quoted_status-text'] = tweet_obj['quoted_status']['text'] 
                tweet_obj['quoted_status-user-screen_name'] = tweet_obj['quoted_status']['user']['screen_name']

            if 'retweeted_status' in tweet_obj:
                tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']
                tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']

            tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']

            tweets.append(tweet_obj)
        except:
            pass
print(len(tweets))

62281


# Prétraitement des tweets
Le nettoyage des tweets comprendra plusieurs choses :

> - Enlever les emojis : pour cela il faut un module Python spécial (si vous connaissez des approches plus simples mettez-les en commentaire ça m’intéresse).
> - Retirer la ponctuation : très facile avec les reg-ex.
> - Retirer les caractères spéciaux : très facile avec les reg-ex mais tous les caractères ne seront pas retirés dans un premier temps. Les tweets sont des objets très sales !.
> - Retirer les chiffres : avec une Reg-ex aussi.
> - Changer les lettres majuscules en minuscules

In [15]:
## create pandas DataFrame for further analysis
import pandas as pd
df_tweet = pd.DataFrame(tweets)


In [16]:
df_tweet.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,display_text_range,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,quoted_status-text,quoted_status-user-screen_name,extended_entities,withheld_in_countries
0,Sun Dec 06 15:00:37 +0000 2020,1335600035569020933,1335600035569020933,RT @SAURAVSINGLA_08: Paper on Transformer Base...,"<a href=""http://derpderp.com"" rel=""nofollow"">L...",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sun Dec 06 15:00:43 +0000 2020,1335600062815199234,1335600062815199234,RT @NumFOCUS: SunPy is a software package for ...,"<a href=""http://www.jacobus.co.uk"" rel=""nofoll...",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sun Dec 06 15:00:44 +0000 2020,1335600067105808389,1335600067105808389,RT @NumFOCUS: SunPy is a software package for ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sun Dec 06 15:00:46 +0000 2020,1335600074810863618,1335600074810863618,RT @Sheraj99: 2021 #AI Policies that organizat...,,False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sun Dec 06 15:00:47 +0000 2020,1335600078791323656,1335600078791323656,"Also this book: \n\n""Happy Git and GitHub for ...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,1.335600e+18,1335600077247836172,388822697.0,388822697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62281 entries, 0 to 62280
Data columns (total 42 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   created_at                         62281 non-null  object 
 1   id                                 62281 non-null  int64  
 2   id_str                             62281 non-null  object 
 3   text                               62281 non-null  object 
 4   source                             62281 non-null  object 
 5   truncated                          62281 non-null  bool   
 6   in_reply_to_status_id              14871 non-null  float64
 7   in_reply_to_status_id_str          14871 non-null  object 
 8   in_reply_to_user_id                15533 non-null  float64
 9   in_reply_to_user_id_str            15533 non-null  object 
 10  in_reply_to_screen_name            15533 non-null  object 
 11  user                               62281 non-null  obj

###### on remarque qu'il ya plusieurs colones dont la valeur est null, cela necessite la supression de ces colones

In [18]:
to_drop = ['in_reply_to_status_id',
           'contributors',
           'geo',
           'withheld_in_countries',
           'place',
           'in_reply_to_status_id_str',
           'in_reply_to_user_id',
           'in_reply_to_user_id_str',
           'in_reply_to_screen_name',
           'coordinates',
           'retweeted_status-user-screen_name',
           'retweeted_status-text',
           'display_text_range',
           'quoted_status_id',
           'quoted_status_id_str',
           'quoted_status',
           'quoted_status_permalink',
           'quoted_status-text',
           'quoted_status-user-screen_name',
           'extended_entities',
           'withheld_in_countries',
           'extended_tweet'
           ]
df_tweet = df_tweet.drop(to_drop, axis=1)

In [19]:
#df_tweet.drop_duplicates()
df_tweet["text"][0]

'RT @SAURAVSINGLA_08: Paper on Transformer Based NLP Models\n\nhttps://t.co/EDizZ954ET\n\n#DataScience #ArtificialIntelligence  #100DaysOfCode #…'

In [20]:
# affichage d'informations statistiques sur les données
df_tweet.describe()

,id,quote_count,reply_count,retweet_count,favorite_count
count,6.228100e+04,62281.0,62281.0,62281.0,62281.0
mean,1.337394e+18,0.0,0.0,0.0,0.0
std,1.134194e+15,0.0,0.0,0.0,0.0
min,1.335600e+18,0.0,0.0,0.0,0.0
25%,1.335627e+18,0.0,0.0,0.0,0.0
50%,1.338121e+18,0.0,0.0,0.0,0.0
75%,1.338122e+18,0.0,0.0,0.0,0.0
max,1.338123e+18,0.0,0.0,0.0,0.0


# Important!!!
### see how to remove the "@.....:" "#.... "

In [21]:
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9_:]+","",str(tweet)) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet
df_tweet['text_clean'] = df_tweet['text'].map(lambda x: cleaner(x))

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [22]:
print(df_tweet['text_clean'])

0               Paper on Transformer Based 100DaysOfCode …
1        is a package for solar physics : a Python - ba...
2        is a package for solar physics : a Python - ba...
3            2021 AI that should lookout for : ️ Machine …
4        Also this book : " Happy Git and for the useR ...
                               ...                        
62276    Happy 31st birthday !!! In honor of her & ; he...
62277                         hey baby i have sex with you
62278          Lucky for you , no one at my Twitter anyway
62279    Continue pressing your on the toilet , for you...
62280       Please let me know what do you prefer or Bulge
Name: text_clean, Length: 62281, dtype: object


In [23]:
print(df_tweet['text'])

0        RT @SAURAVSINGLA_08: Paper on Transformer Base...
1        RT @NumFOCUS: SunPy is a software package for ...
2        RT @NumFOCUS: SunPy is a software package for ...
3        RT @Sheraj99: 2021 #AI Policies that organizat...
4        Also this book: \n\n"Happy Git and GitHub for ...
                               ...                        
62276    Happy 31st birthday @taylorswift13!!! In honor...
62277          @BestActress4  hey baby i have sex with you
62278     Lucky for you, no one looks at my Twitter anyway
62279    RT @aproko_doctor: Continue pressing your phon...
62280    RT @xxxSM_Girls: Please let me know what do yo...
Name: text, Length: 62281, dtype: object


In [24]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df_tweet['Tweet_tokenized'] = df_tweet['text_clean'].apply(lambda x: tokenization(x.lower()))
print(df_tweet['Tweet_tokenized'])

0         [paper, on, transformer, based, 100daysofcode, ]
1        [is, a, package, for, solar, physics, a, pytho...
2        [is, a, package, for, solar, physics, a, pytho...
3        [2021, ai, that, should, lookout, for, machine, ]
4        [also, this, book, happy, git, and, for, the, ...
                               ...                        
62276    [happy, 31st, birthday, in, honor, of, her, he...
62277                 [hey, baby, i, have, sex, with, you]
62278    [lucky, for, you, no, one, at, my, twitter, an...
62279    [continue, pressing, your, on, the, toilet, fo...
62280    [please, let, me, know, what, do, you, prefer,...
Name: Tweet_tokenized, Length: 62281, dtype: object


In [25]:
%load_ext watermark


The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [26]:
%watermark --iversions

json  : 2.0.9
emoji : 0.6.0
sys   : 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
re    : 2.2.1
pandas: 1.0.5
nltk  : 3.5

